# Goal:

- provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

- Used as a Python sanity check because I'm not that familiar with R.

In [2]:
import numpy as np
np.random.seed(42)
import pandas as pd
import tqdm
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from numba import njit

np.seterr(all='raise')

{'divide': 'raise', 'over': 'raise', 'under': 'raise', 'invalid': 'raise'}

# Load and Process the Data

In [7]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../train_data/X_norm_1024.csv"
    y_file = "../train_data/y_1024.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)

    class_counts = train_y.iloc[:, 0].value_counts()
    print(class_counts)

    # If it's a binary classification, calculate the ratio
    positive_class = class_counts[1.0]  
    negative_class = class_counts[0.0]  
    ratio = positive_class / negative_class
    
    print(f"Positive class: {positive_class}, Negative class: {negative_class}, Ratio: {ratio:.2f}")
      
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [12]:
NUM_SAMPLES = -1
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)

# Same shape as Marcelo's reference code
betas = np.zeros((train_x.shape[1], ))

IMORT
1    512
0    512
Name: count, dtype: int64
Positive class: 512, Negative class: 512, Ratio: 1.00
Using subsampled data to compare Python-C++
Reading in ../train_data/X_norm_1024.csv, ../train_data/y_1024.csv 
Train X shape is: (1023, 10)
Train y shape is: (1023, 1)


In [13]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas   # A vector of linear_predictions/logits z = train_x @ weights
    if dbg:
        print(f"Logits: {preds}")
    return np.expand_dims(sigmoid(preds), -1)   # Shape: (m, 1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)   # A vector of logistic predictions y_hat = sigmoid(z)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient   # Shape: (10, 1) == Rows correspond to num_features
    ## This function is used to update the values of betas: w_new = w_old + lr * gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))   # h: hypothesis, basically preds/y_hat
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    ## np.clip() function prevents computing the log of 0, by taking the minimum of 1e-15.
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]   # Shape: (1,) == scalar value

def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    for i in tqdm.trange(epochs):
    # for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)

        ## Assign updated weights into phi_prime
        phi_prime = theta - lr * np.squeeze(gradient)   # np.squeeze() removes single dimensions --> shape (10,)
        
        ## Nesterov acceleration process
        if i == 0:
            theta = phi_prime
        else:
            ## If current updated weight (phi_prime) < previous weight (phi), 
            ## The updated weight theta will be even smaller.
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime   # phi is then the weight of the previous epoch/update
        loss = cost(train_x, train_y, theta)
        nesterov_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return nesterov_loss, theta, phi


In [14]:
losses, theta, phi = nesterov(betas, 200, lr, mu, train_x, train_y)

100%|██████████| 200/200 [00:00<00:00, 2444.08it/s]


In [15]:
losses

## zero initialized betas    : cost = 0.4087381183990924
## uniform initialized betas : cost = 0.40867096414830645
## randn initialized betas   : cost = 0.40850863604124726

[np.float64(0.682438146439526),
 np.float64(0.6714453914072441),
 np.float64(0.6611013485147301),
 np.float64(0.6514162146586544),
 np.float64(0.6423259325233592),
 np.float64(0.6337707531714274),
 np.float64(0.6257002096176623),
 np.float64(0.6180716210017204),
 np.float64(0.6108484558248058),
 np.float64(0.6039990483572646),
 np.float64(0.5974956250378402),
 np.float64(0.5913135670891185),
 np.float64(0.5854308495746274),
 np.float64(0.5798276116655406),
 np.float64(0.5744858243802814),
 np.float64(0.5693890307953972),
 np.float64(0.5645221402767862),
 np.float64(0.5598712631529988),
 np.float64(0.5554235758543887),
 np.float64(0.5511672091916684),
 np.float64(0.5470911543892155),
 np.float64(0.5431851829072069),
 np.float64(0.5394397771210908),
 np.float64(0.5358460696801214),
 np.float64(0.532395789914908),
 np.float64(0.5290812160631745),
 np.float64(0.525895132374097),
 np.float64(0.5228307903645797),
 np.float64(0.5198818736572505),
 np.float64(0.5170424659455463),
 np.float64(0

In [16]:
theta

array([-1.24842111,  0.19234508, -1.19103479,  0.43030586, -0.64449037,
       -0.04041207, -0.0847023 , -0.36362034,  0.11810004,  0.44677751])

In [17]:
pred = fwd(train_x, theta, dbg=False)

## Decision (Threshold = 0.5)
train_y_hat = (pred >= 0.5).astype(int)

In [18]:
train_y_hat.shape

(1023, 1)

In [19]:
train_y.shape

(1023, 1)

In [20]:
confusion_matrix(train_y, train_y_hat)

array([[463,  48],
       [136, 376]])

In [21]:
roc_auc_score(train_y, pred)

np.float64(0.8901472296966731)

In [22]:
accuracy_score(train_y, train_y_hat)

0.820136852394917

In [23]:
f1_score(train_y, train_y_hat)

0.8034188034188035

On test data


In [ ]:
log_loss(train_y, pred)